In [1]:
import os
from typing import Annotated

from typing_extensions import TypedDict
from dotenv import load_dotenv
from IPython.display import Image, display

from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langsmith import utils 
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from agents.agent_tool import DataTools
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
import yfinance as yf
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages
# 환경 변수 로드
load_dotenv()

utils.tracing_is_enabled()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
FRED_API_KEY = os.getenv("FRED_API_KEY")
print(len(FRED_API_KEY))

32


In [3]:
import requests

# API 키 설정 (여기에 본인의 API 키를 입력)
FRED_API_KEY = os.getenv("FRED_API_KEY")
# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/tags"

# 파라미터 설정
params = {
    "api_key": FRED_API_KEY,
    "file_type": "json",  # JSON 형식으로 응답 요청
    "search_text": "cpi",  # "cpi" 태그 검색
    "limit": 10,  # 최대 10개의 결과 반환
    "order_by": "series_count",  # 데이터 시리즈 개수 기준 정렬
    "sort_order": "desc"  # 내림차순 정렬
}

# API 요청
response = requests.get(BASE_URL, params=params)

# 응답 확인
if response.status_code == 200:
    data = response.json()  # JSON 데이터 변환
    print(data)  # 응답 출력
else:
    print(f"Error: {response.status_code}, {response.text}")


{'realtime_start': '2025-02-20', 'realtime_end': '2025-02-20', 'order_by': 'series_count', 'sort_order': 'desc', 'count': 1, 'offset': 0, 'limit': 10, 'tags': [{'name': 'cpi', 'group_id': 'gen', 'notes': 'Consumer Price Index', 'created': '2012-02-27 10:18:19-06', 'popularity': 68, 'series_count': 20822}]}


In [6]:
import requests

# FRED API 키 입력 (본인의 API 키로 대체)

# FRED API URL (tags/series)
BASE_URL = "https://api.stlouisfed.org/fred/tags/series"

# API 요청 파라미터 설정
params = {
    "api_key": FRED_API_KEY,  # FRED API 키
    "file_type": "json",  # JSON 형식으로 응답 요청
    "tag_names": "cpi",  # "CPI" 태그를 가진 데이터 시리즈 검색
    "limit": 10,  # 최대 10개의 시리즈 가져오기
    "order_by": "last_updated",  # 인기순 정렬
    "sort_order": "desc"  # 내림차순 정렬
}

# API 요청 실행
response = requests.get(BASE_URL, params=params)

# 응답 확인 및 출력
if response.status_code == 200:
    data = response.json()
    print(data)  # JSON 데이터 출력
else:
    print(f"Error: {response.status_code}, {response.text}")


{'realtime_start': '2025-02-20', 'realtime_end': '2025-02-20', 'order_by': 'last_updated', 'sort_order': 'desc', 'count': 20838, 'offset': 0, 'limit': 10, 'seriess': [{'id': 'ARGCP120000GYM', 'realtime_start': '2025-02-20', 'realtime_end': '2025-02-20', 'title': 'Consumer Price Indices (CPIs, HICPs), COICOP 1999: Consumer Price Index: Miscellaneous Goods and Services for Argentina', 'observation_start': '2017-12-01', 'observation_end': '2025-01-01', 'frequency': 'Monthly', 'frequency_short': 'M', 'units': 'Growth rate same period previous year', 'units_short': 'Growth rate same period previous Yr.', 'seasonal_adjustment': 'Not Seasonally Adjusted', 'seasonal_adjustment_short': 'NSA', 'last_updated': '2025-02-17 14:22:49-06', 'popularity': 0, 'group_popularity': 1, 'notes': 'OECD Data Filters: \nREF_AREA: ARG\nMEASURE: CPI\nUNIT_MEASURE: PA\nMETHODOLOGY: N\nEXPENDITURE: CP12\nADJUSTMENT: N\nTRANSFORMATION: GY\nFREQ: M\n\nAll OECD data should be cited as follows: OECD (year), (dataset na

In [7]:
import requests



# 미국 CPI 시리즈 ID (전국 소비자물가지수, 도시 지역 기준, 모든 품목)
SERIES_ID = "CPIAUCSL"  # Consumer Price Index for All Urban Consumers

# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

# API 요청 파라미터 설정
params = {
    "api_key": FRED_API_KEY,         # FRED API 키
    "file_type": "json",        # JSON 형식으로 응답 요청
    "series_id": SERIES_ID,     # 특정 시리즈 ID 지정
    "observation_start": "2024-01-01",  # 조회 시작일 (예: 2024년 1월)
    "observation_end": "2025-01-01",    # 조회 종료일 (예: 2025년 1월)
    "sort_order": "desc",       # 최신 데이터가 먼저 나오도록 정렬
    "limit": 12                 # 최근 12개월 데이터 가져오기
}

# API 호출
response = requests.get(BASE_URL, params=params)

# 응답 확인 및 출력
if response.status_code == 200:
    data = response.json()
    
    # 관측된 CPI 데이터 출력
    print("미국 CPI 데이터 (최근 12개월):")
    for obs in data["observations"]:
        print(f"날짜: {obs['date']}, CPI 값: {obs['value']}")
else:
    print(f"Error: {response.status_code}, {response.text}")


미국 CPI 데이터 (최근 12개월):
날짜: 2025-01-01, CPI 값: 319.086
날짜: 2024-12-01, CPI 값: 317.603
날짜: 2024-11-01, CPI 값: 316.449
날짜: 2024-10-01, CPI 값: 315.564
날짜: 2024-09-01, CPI 값: 314.851
날짜: 2024-08-01, CPI 값: 314.131
날짜: 2024-07-01, CPI 값: 313.566
날짜: 2024-06-01, CPI 값: 313.131
날짜: 2024-05-01, CPI 값: 313.140
날짜: 2024-04-01, CPI 값: 313.016
날짜: 2024-03-01, CPI 값: 312.107
날짜: 2024-02-01, CPI 값: 311.022


In [48]:
import requests


# 미국 CPI 및 근원 CPI 시리즈 ID
CPI_ID = "CPIAUCSL"  # 미국 일반 CPI
CORE_CPI_ID = "CPILFESL"  # 미국 근원 CPI (식료품·에너지 제외)

# CPI 데이터의 Release ID (CPI는 Release ID 10번)
RELEASE_ID = 10  # CPI 발표 일정

# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"
RELEASE_URL = "https://api.stlouisfed.org/fred/release/dates"

#  CPI 데이터 가져오기 (전년 동월 대비 YoY%, 전월 대비 MoM% 포함)
def get_cpi_data(series_id, units=None):
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "series_id": series_id,
        "observation_start": "2024-01-01",
        "observation_end": "2025-01-01",
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 데이터 조회
    }
    
    if units:  # 변환 옵션 적용 (YoY 또는 MoM)
        params["units"] = units
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        return response.json()["observations"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

#  CPI 발표 일정 가져오기
def get_cpi_release_dates():
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "release_id": RELEASE_ID,
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 발표 일정 가져오기
    }
    response = requests.get(RELEASE_URL, params=params)
    
    if response.status_code == 200:
        return [release["date"] for release in response.json()["release_dates"]]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

#  데이터 가져오기 (CPI 값, YoY, MoM)
cpi_data = get_cpi_data(CPI_ID)  # 일반 CPI 값
cpi_yoy_data = get_cpi_data(CPI_ID, units="pc1")  # 일반 CPI 전년 동월 대비 상승률(%)
cpi_mom_data = get_cpi_data(CPI_ID, units="pch")  # 일반 CPI 전월 대비 상승률(%)

core_cpi_data = get_cpi_data(CORE_CPI_ID)  # 근원 CPI 값
core_cpi_yoy_data = get_cpi_data(CORE_CPI_ID, units="pc1")  # 근원 CPI 전년 동월 대비 상승률(%)
core_cpi_mom_data = get_cpi_data(CORE_CPI_ID, units="pch")  # 근원 CPI 전월 대비 상승률(%)

release_dates = get_cpi_release_dates()  # CPI 발표 일정 가져오기

#  발표 일정 개수가 부족하면 "N/A"로 채우기
while len(release_dates) < len(cpi_data):
    release_dates.append("N/A")  # 부족한 발표 일정은 "N/A" 처리

# 데이터 출력 (CPI + 발표일정 포함)
print("\n 미국 CPI 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(cpi_data)):
    print(f"데이터 기준 월: {cpi_data[i]['date']}, CPI: {cpi_data[i]['value']}, CPI YoY: {cpi_yoy_data[i]['value']}%, CPI MoM: {cpi_mom_data[i]['value']}%, 발표일: {release_dates[i]}")

print("\n 미국 근원 CPI 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(core_cpi_data)):
    print(f"데이터 기준 월: {core_cpi_data[i]['date']}, 근원 CPI: {core_cpi_data[i]['value']}, 근원 CPI YoY: {core_cpi_yoy_data[i]['value']}%, 근원 CPI MoM: {core_cpi_mom_data[i]['value']}%, 발표일: {release_dates[i]}")



 미국 CPI 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)
데이터 기준 월: 2025-01-01, CPI: 319.086, CPI YoY: 2.99941%, CPI MoM: 0.46694%, 발표일: 2025-02-12
데이터 기준 월: 2024-12-01, CPI: 317.603, CPI YoY: 2.87237%, CPI MoM: 0.36467%, 발표일: 2025-01-15
데이터 기준 월: 2024-11-01, CPI: 316.449, CPI YoY: 2.71417%, CPI MoM: 0.28045%, 발표일: 2024-12-11
데이터 기준 월: 2024-10-01, CPI: 315.564, CPI YoY: 2.57140%, CPI MoM: 0.22646%, 발표일: 2024-11-13
데이터 기준 월: 2024-09-01, CPI: 314.851, CPI YoY: 2.43254%, CPI MoM: 0.22920%, 발표일: 2024-10-10
데이터 기준 월: 2024-08-01, CPI: 314.131, CPI YoY: 2.61091%, CPI MoM: 0.18019%, 발표일: 2024-09-11
데이터 기준 월: 2024-07-01, CPI: 313.566, CPI YoY: 2.93846%, CPI MoM: 0.13892%, 발표일: 2024-08-14
데이터 기준 월: 2024-06-01, CPI: 313.131, CPI YoY: 2.97009%, CPI MoM: -0.00287%, 발표일: 2024-07-11
데이터 기준 월: 2024-05-01, CPI: 313.140, CPI YoY: 3.23887%, CPI MoM: 0.03961%, 발표일: 2024-06-12
데이터 기준 월: 2024-04-01, CPI: 313.016, CPI YoY: 3.35405%, CPI MoM: 0.29125%, 발표일: 2024-05-15
데이터 기준 월: 2024-03-01, CPI: 312.107, CPI YoY

In [26]:
import requests

# FRED API 키 입력
API_KEY = "your_correct_32_character_api_key"

# 미국 PCE 및 근원 PCE 시리즈 ID
PCE_ID = "PCEPI"  # 전체 PCE (Headline PCE)
CORE_PCE_ID = "PCEPILFE"  # 근원 PCE (Core PCE, 식료품·에너지 제외)

# PCE 데이터의 Release ID (PCE는 Release ID 225번)
RELEASE_ID = 225  # PCE 발표 일정 ID

# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"
RELEASE_URL = "https://api.stlouisfed.org/fred/release/dates"

# 📌 1️⃣ PCE 데이터 가져오기 (전년 동월 대비 YoY%, 전월 대비 MoM% 포함)
def get_pce_data(series_id, units=None):
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "series_id": series_id,
        "observation_start": "2024-01-01",
        "observation_end": "2025-01-01",
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 데이터 조회
    }
    
    if units:  # 변환 옵션 적용 (YoY 또는 MoM)
        params["units"] = units
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        return response.json()["observations"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

# 📌 2️⃣ PCE 발표 일정 가져오기
def get_pce_release_dates():
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "release_id": RELEASE_ID,
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 발표 일정 가져오기
    }
    response = requests.get(RELEASE_URL, params=params)
    
    if response.status_code == 200:
        return [release["date"] for release in response.json()["release_dates"]]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

# 📌 데이터 가져오기 (PCE 값, YoY, MoM)
pce_data = get_pce_data(PCE_ID)  # 전체 PCE 값
pce_yoy_data = get_pce_data(PCE_ID, units="pc1")  # 전체 PCE 전년 동월 대비 상승률(%)
pce_mom_data = get_pce_data(PCE_ID, units="pch")  # 전체 PCE 전월 대비 상승률(%)

core_pce_data = get_pce_data(CORE_PCE_ID)  # 근원 PCE 값
core_pce_yoy_data = get_pce_data(CORE_PCE_ID, units="pc1")  # 근원 PCE 전년 동월 대비 상승률(%)
core_pce_mom_data = get_pce_data(CORE_PCE_ID, units="pch")  # 근원 PCE 전월 대비 상승률(%)

release_dates = get_pce_release_dates()  # PCE 발표 일정 가져오기

# 📌 발표 일정 개수가 부족하면 "N/A"로 채우기
while len(release_dates) < len(pce_data):
    release_dates.append("N/A")  # 부족한 발표 일정은 "N/A" 처리

# 📌 데이터 출력 (PCE + 발표일정 포함)
print("\n📌 미국 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(pce_data)):
    print(f"데이터 기준 월: {pce_data[i]['date']}, PCE: {pce_data[i]['value']}, PCE YoY: {pce_yoy_data[i]['value']}%, PCE MoM: {pce_mom_data[i]['value']}%, 발표일: {release_dates[i]}")

print("\n📌 미국 근원 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(core_pce_data)):
    print(f"데이터 기준 월: {core_pce_data[i]['date']}, 근원 PCE: {core_pce_data[i]['value']}, 근원 PCE YoY: {core_pce_yoy_data[i]['value']}%, 근원 PCE MoM: {core_pce_mom_data[i]['value']}%, 발표일: {release_dates[i]}")



📌 미국 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)
데이터 기준 월: 2024-12-01, PCE: 124.705, PCE YoY: 2.55177%, PCE MoM: 0.25565%, 발표일: 2017-10-25
데이터 기준 월: 2024-11-01, PCE: 124.387, PCE YoY: 2.44780%, PCE MoM: 0.12235%, 발표일: 2017-07-26
데이터 기준 월: 2024-10-01, PCE: 124.235, PCE YoY: 2.31756%, PCE MoM: 0.23883%, 발표일: 2017-04-25
데이터 기준 월: 2024-09-01, PCE: 123.939, PCE YoY: 2.10237%, PCE MoM: 0.17134%, 발표일: 2017-01-26
데이터 기준 월: 2024-08-01, PCE: 123.727, PCE YoY: 2.28331%, PCE MoM: 0.12300%, 발표일: 2016-10-26
데이터 기준 월: 2024-07-01, PCE: 123.575, PCE YoY: 2.46853%, PCE MoM: 0.16698%, 발표일: 2016-07-26
데이터 기준 월: 2024-06-01, PCE: 123.369, PCE YoY: 2.43617%, PCE MoM: 0.11767%, 발표일: 2016-04-25
데이터 기준 월: 2024-05-01, PCE: 123.224, PCE YoY: 2.56701%, PCE MoM: -0.00811%, 발표일: 2016-01-27
데이터 기준 월: 2024-04-01, PCE: 123.234, PCE YoY: 2.72068%, PCE MoM: 0.26198%, 발표일: 2015-10-26
데이터 기준 월: 2024-03-01, PCE: 122.912, PCE YoY: 2.80963%, PCE MoM: 0.34124%, 발표일: 2015-06-23
데이터 기준 월: 2024-02-01, PCE: 122.494, PCE Yo

In [20]:
core_ppi_yoy_data

[]

In [ ]:
import requests

# FRED API 키 입력
API_KEY = "your_correct_32_character_api_key"

# 미국 PCE 및 근원 PCE 시리즈 ID
PCE_ID = "PCEPI"  # 전체 PCE (Headline PCE)
CORE_PCE_ID = "PCEPILFE"  # 근원 PCE (Core PCE, 식료품·에너지 제외)

# 앞서 확인한 정확한 PCE 데이터의 Release ID 입력
RELEASE_ID = 54  # 예시로 82를 사용하였으며, 실제 확인한 Release ID로 변경 필요

# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"
RELEASE_DATES_URL = "https://api.stlouisfed.org/fred/release/dates"

# 📌 1️⃣ PCE 데이터 가져오기 (전년 동월 대비 YoY%, 전월 대비 MoM% 포함)
def get_pce_data(series_id, units=None):
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "series_id": series_id,
        "observation_start": "2024-01-01",
        "observation_end": "2025-01-01",
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 데이터 조회
    }
    
    if units:  # 변환 옵션 적용 (YoY 또는 MoM)
        params["units"] = units
    
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        return response.json()["observations"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

#  PCE 발표 일정 가져오기
def get_pce_release_dates():
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "release_id": RELEASE_ID,
        "sort_order": "desc",
        "limit": 12  # 최근 12개월 발표 일정 가져오기
    }
    response = requests.get(RELEASE_DATES_URL, params=params)
    # return response
    if response.status_code == 200:
        return [release["date"] for release in response.json()["release_dates"]]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

# 📌 데이터 가져오기 (PCE 값, YoY, MoM)
pce_data = get_pce_data(PCE_ID)  # 전체 PCE 값
pce_yoy_data = get_pce_data(PCE_ID, units="pc1")  # 전체 PCE 전년 동월 대비 상승률(%)
pce_mom_data = get_pce_data(PCE_ID, units="pch")  # 전체 PCE 전월 대비 상승률(%)

core_pce_data = get_pce_data(CORE_PCE_ID)  # 근원 PCE 값
core_pce_yoy_data = get_pce_data(CORE_PCE_ID, units="pc1")  # 근원 PCE 전년 동월 대비 상승률(%)
core_pce_mom_data = get_pce_data(CORE_PCE_ID, units="pch")  # 근원 PCE 전월 대비 상승률(%)

release_dates = get_pce_release_dates()  # PCE 발표 일정 가져오기

# 📌 발표 일정 개수가 부족하면 "N/A"로 채우기
while len(release_dates) < len(pce_data):
    release_dates.append("N/A")  # 부족한 발표 일정은 "N/A" 처리

# 📌 데이터 출력 (PCE + 발표일정 포함)
print("\n📌 미국 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(pce_data)):
    print(f"데이터 기준 월: {pce_data[i]['date']}, PCE: {pce_data[i]['value']}, PCE YoY: {pce_yoy_data[i]['value']}%, PCE MoM: {pce_mom_data[i]['value']}%, 발표일: {release_dates[i]}")

print("\n📌 미국 근원 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)")
for i in range(len(core_pce_data)):
    print(f"데이터 기준 월: {core_pce_data[i]['date']}, PCE: {core_pce_data[i]['value']}, PCE YoY: {core_pce_yoy_data[i]['value']}%, PCE MoM: {core_pce_mom_data[i]['value']}%, 발표일: {release_dates[i]}")

 



📌 미국 PCE 및 전년 동월 대비(YoY %) & 전월 대비(MoM %) 변화율 (최근 12개월)
데이터 기준 월: 2024-12-01, PCE: 124.705, PCE YoY: 2.55177%, PCE MoM: 0.25565%, 발표일: 2025-01-31
데이터 기준 월: 2024-11-01, PCE: 124.387, PCE YoY: 2.44780%, PCE MoM: 0.12235%, 발표일: 2024-12-20
데이터 기준 월: 2024-10-01, PCE: 124.235, PCE YoY: 2.31756%, PCE MoM: 0.23883%, 발표일: 2024-11-27
데이터 기준 월: 2024-09-01, PCE: 123.939, PCE YoY: 2.10237%, PCE MoM: 0.17134%, 발표일: 2024-10-31
데이터 기준 월: 2024-08-01, PCE: 123.727, PCE YoY: 2.28331%, PCE MoM: 0.12300%, 발표일: 2024-09-27
데이터 기준 월: 2024-07-01, PCE: 123.575, PCE YoY: 2.46853%, PCE MoM: 0.16698%, 발표일: 2024-08-30
데이터 기준 월: 2024-06-01, PCE: 123.369, PCE YoY: 2.43617%, PCE MoM: 0.11767%, 발표일: 2024-07-26
데이터 기준 월: 2024-05-01, PCE: 123.224, PCE YoY: 2.56701%, PCE MoM: -0.00811%, 발표일: 2024-06-28
데이터 기준 월: 2024-04-01, PCE: 123.234, PCE YoY: 2.72068%, PCE MoM: 0.26198%, 발표일: 2024-05-31
데이터 기준 월: 2024-03-01, PCE: 122.912, PCE YoY: 2.80963%, PCE MoM: 0.34124%, 발표일: 2024-04-26
데이터 기준 월: 2024-02-01, PCE: 122.494, PCE Yo

In [34]:
release_dates.content

b'{"realtime_start":"1776-07-04","realtime_end":"9999-12-31","order_by":"release_date","sort_order":"desc","count":27,"offset":0,"limit":12,"release_dates":[{"release_id":82,"date":"2024-03-21"},{"release_id":82,"date":"2023-03-20"},{"release_id":82,"date":"2022-04-18"},{"release_id":82,"date":"2020-02-20"},{"release_id":82,"date":"2019-03-19"},{"release_id":82,"date":"2018-02-12"},{"release_id":82,"date":"2017-06-29"},{"release_id":82,"date":"2016-03-11"},{"release_id":82,"date":"2015-02-02"},{"release_id":82,"date":"2014-03-14"},{"release_id":82,"date":"2013-04-10"},{"release_id":82,"date":"2012-10-24"}]}'

In [ ]:
import requests


# FRED API URL
RELEASES_DATES_URL = "https://api.stlouisfed.org/fred/releases/dates"

#  FRED 발표 일정 데이터 가져오기
def get_all_release_dates():
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "realtime_start": "2024-01-01",
        "realtime_end": "9999-12-31",
        "limit": 1000,  # 최대 1000개 데이터 가져오기
        "order_by": "release_date",
        "sort_order": "desc"
    }
    response = requests.get(RELEASES_DATES_URL, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# 실행
release_dates = get_all_release_dates()

# 결과 출력
if release_dates:
    for release in release_dates["release_dates"]:
        print(f"Release Date: {release['date']}, Release ID: {release['release_id']}, Release Name: {release['release_name']}")


Release Date: 2025-02-20, Release ID: 319, Release Name: BIS Effective Exchange Rate Indices
Release Date: 2025-02-20, Release ID: 221, Release Name: Chicago Fed National Financial Conditions Index
Release Date: 2025-02-20, Release ID: 441, Release Name: Coinbase Cryptocurrencies
Release Date: 2025-02-20, Release ID: 86, Release Name: Commercial Paper
Release Date: 2025-02-20, Release ID: 72, Release Name: Daily Treasury Inflation-Indexed Securities
Release Date: 2025-02-20, Release ID: 279, Release Name: Economic Policy Uncertainty
Release Date: 2025-02-20, Release ID: 502, Release Name: Euro Short Term Rate
Release Date: 2025-02-20, Release ID: 378, Release Name: Federal Funds Data
Release Date: 2025-02-20, Release ID: 101, Release Name: FOMC Press Release
Release Date: 2025-02-20, Release ID: 18, Release Name: H.15 Selected Interest Rates
Release Date: 2025-02-20, Release ID: 20, Release Name: H.4.1 Factors Affecting Reserve Balances
Release Date: 2025-02-20, Release ID: 504, Releas

In [50]:
import requests

# FRED API 키 입력
API_KEY = "your_correct_32_character_api_key"

# 미시간 대학교 기대 인플레이션 시리즈 ID
MICH_EXP_INFLATION_ID = "MICH"

# FRED API URL
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

# 📌 API 요청 함수 (최신 데이터 포함)
def get_michigan_inflation_data():
    params = {
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "series_id": MICH_EXP_INFLATION_ID,
        "observation_start": "2024-01-01",  # 시작일 유지
        "observation_end": "2025-02-01",  # 최신 데이터 포함하도록 변경
        "sort_order": "desc",  # 최신 데이터 먼저 가져오도록 변경
        "limit": 100  # 충분한 데이터 확보
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        return response.json()["observations"]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

#  데이터 가져오기
mich_data = get_michigan_inflation_data()

#  보기 쉽게 출력
print("\n 미시간 대학교 소비자 인플레이션 기대치\n")
print(f"{'날짜':<12} {'기대 인플레이션 (%)':<10}")
print("=" * 30)

for entry in mich_data:
    date = entry["date"]
    value = entry["value"]
    print(f"{date:<12} {value:<10}")


 미시간 대학교 소비자 인플레이션 기대치

날짜           기대 인플레이션 (%)
2024-12-01   2.8       
2024-11-01   2.6       
2024-10-01   2.7       
2024-09-01   2.7       
2024-08-01   2.8       
2024-07-01   2.9       
2024-06-01   3         
2024-05-01   3.3       
2024-04-01   3.2       
2024-03-01   2.9       
2024-02-01   3         
2024-01-01   2.9       
